In [ ]:
# this is an example of a encoder & decoder network. The purpose of this code is to show a typical encoder-decoder
#  structure. I hide/modify the attention part since students will need to implement it in the hw4. 

# Do not copy the attention part of this code, it's not the right version for hw.
# Do not copy the attention part of this code, it's not the right version for hw.
# Do not copy the attention part of this code, it's not the right version for hw.


class MynetWork(nn.Module):
    def __init__(self, src_vocab, embedding_size, output_size, target_vocav):
        super().__init__()
        self.encoder = Encoder(src_vocab, embedding_size)
        self.decoder = Decoder(trg_vocab, output_size)
        self.out = nn.Linear(output_size, trg_vocab)
    def forward(self, src, trg, src_mask, target_vocav):
        e_outputs = self.encoder(src)
        d_output = self.decoder(e_outputs)
        output = self.out(d_output)
        return output

class Eecoder(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(Listener, self).__init__()

        self.pblstm1 = pBLSTM(input_dim, hidden_dim)
        self.pblstm2 = pBLSTM(hidden_dim*2, hidden_dim)
        self.pblstm3 = pBLSTM(hidden_dim*2, hidden_dim)
        self.dropout2 = nn.Dropout(p=0.3)

        self.mlp1 = MLP(hidden_dim*2, 256, 128)
        self.mlp2 = MLP(hidden_dim*2, 256, 128)


    def forward(self, x):

        x = self.pblstm1(x)
        x = self.pblstm2(x)
        x = self.pblstm3(x)
        x = self.dropout2(x)  
        
        keys = self.mlp1(x)
        values = self.mlp2(x)       
        return keys, values

class Decoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, vocab_size):
        super(Listener, self).__init__()

        self.pblstm1 = pBLSTM(input_dim, hidden_dim)
        self.pblstm2 = pBLSTM(hidden_dim*2, hidden_dim)
        self.pblstm3 = pBLSTM(hidden_dim*2, hidden_dim)
        self.embedding = nn.Embedding(vocab_size, hidden_dim)
        self.dropout2 = nn.Dropout(p=0.3)
        self.mlp1 = MLP(hidden_dim*2, 256, 128)
        self.myAttention = Attention()

    def forward(self, keys, values, y):
        
        result = []
        
        context = Attention(keys, values)
        
        # y will the target seuquence 
        embedding_y = self.embedding(y)
        
        for i in range(y.shape[1] - 1):
            embedding_yi = embedding_y[:,i,:]
            x = torch.cat((embedding_yi, context), dim=1)
            
            # x = embedding_yi + context
            x = self.pblstm1(x)
            x = self.pblstm2(x)
            x = self.pblstm3(x)
            x = self.dropout2(x)
            output = self.mlp1(x)
            result.append(output) 
            
        result = np.asarray(result)
        return result


class pBLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(Encoder, self).__init__()
        self.blstm = nn.LSTM(input_dim*2, hidden_dim, 1, batch_first=True, bidirectional=True)

    def forward(self, x):

        # reduce the timestep
        x = x.contiguous().view(batch_size, int(seq_length//2), feature_dim*2)
        output, _ = self.blstm(x)
        return output
    
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x